# Load Zarr Image with labels from a public S3 repository, analyze using ilastik and compare results.
The notebook shows how to load an IDR image converted into a Zarr file with labels.

The image is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247).

This original image was converted into the Zarr format. The analysis results produced by the paper authors were converted into labels and linked to the Zarr file which was placed into a public S3 repository.

In this notebook, the converted Zarr file is then loaded together with the labels from the S3 storage and analyzed using [ilastik](https://ilastik.github.io/). The ilastik analysis produces a probability map based on pixel classification. These probability maps are then viewed side-by-side with the original segmentations produced by the authors of the papers obtained via the loaded labels.

If you run this notebook with different images or ilastik projects, the dimension might need to be adjusted depending on the ilastik project.

### Insert required packages

In [1]:
import os
import numpy
import zarr
import dask.array as da
from dask.diagnostics import ProgressBar

from ilastik import app
from ilastik.applets.dataSelection.opDataSelection import PreloadedArrayDatasetInfo
import vigra

# package for 3d visualization
from itkwidgets import compare, view

/srv/conda/envs/notebook/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


### Enter the image ID

In [2]:
image_id = 6001247

### Helper method to load an Image as 5D-numpy array from S3

In [3]:
def load_from_s3(id, resolution='0'):
    endpoint_url = 'https://s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    # data.shape is (t, c, z, y, x) by convention
    with ProgressBar():
        return numpy.asarray(da.from_zarr(endpoint_url + root))

### Helper method to load the labels linked to the Image from S3 

In [4]:
def load_labels_from_s3(id, resolution='0'):
    endpoint_url = 'https://s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/labels/%s/' % (id, resolution)
    return da.from_zarr(endpoint_url + root)

### Load the image data

In [5]:
%time input_data = load_from_s3(image_id)

[########################################] | 100% Completed | 15.8s
CPU times: user 3.11 s, sys: 1.71 s, total: 4.81 s
Wall time: 16.2 s


### Load the labels

In [6]:
labels = load_labels_from_s3(image_id)

### Store the original image channels for later comparison with probability map generated by ilastik.

In [7]:
first_channel = input_data[0, 0, :, : ,:]
second_channel = input_data[0, 1, :, : ,:]

### Load each image as a 5D-numpy array and analyze in ilastik.

In [8]:
# Re-order the array tczyx -> tzyxc
print(input_data.shape)
input_data = input_data.swapaxes(1, 2).swapaxes(2, 3).swapaxes(3, 4)
print(input_data.shape)

(1, 2, 257, 210, 253)
(1, 257, 210, 253, 2)


In [9]:
# Load the model linked to the dataset

home = os.path.expanduser("~")
model_file = home+"/notebooks/pipelines/pixel-class-133.ilp"


# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = app.parse_args([])
args.headless = True
args.project = model_file
shell = app.main(args)

print('running ilastik using %s' % model_file)
role_data_dict = [ {"Raw Data": PreloadedArrayDatasetInfo(preloaded_array=input_data, axistags=vigra.defaultAxistags("tzyxc"))}]
predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
for data in predictions:
    # Re-organise array from tzyxc to tczyx order
    data = data.swapaxes(4, 3).swapaxes(3, 2).swapaxes(2, 1)
print("done")

INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".
Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".
INFO ilastik.app: Resetting lazyflow thread pool with 2 threads.
INFO ilastik.app: Configuring lazyflow RAM limit to 2.0GiB
INFO lazyflow.utility.memory: Available memory set to 2.0GiB


WARNING 2021-04-29 11:20:52,600 memory 93 140039328896832 User specified memory exceeds memory physically available. Please check the configuration.
WARNING 2021-04-29 11:20:53,373 opConservationTracking 93 140039328896832 Could not find any ILP solver
WARNING 2021-04-29 11:20:53,396 opStructuredTracking 93 140039328896832 Could not find any ILP solver
WARNING 2021-04-29 11:20:53,400 structuredTrackingWorkflow 93 140039328896832 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!
WARNING 2021-04-29 11:20:53,840 __init__ 93 140039328896832 Failed to import NeuralNet workflow; check dependencies: No module named 'PyQt5'
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/ilastik-meta/ilastik/ilastik/workflows/__init__.py", line 137, in <module>
    from .nnClassification import NNClassificationWorkflow
  File "/srv/conda/envs/notebook/ilastik-meta/ilastik/ilastik/workflows/nnClassification/__init__.py", l

INFO ilastik.shell.projectManager: Opening Project: /home/jmarie/notebooks/pipelines/pixel-class-133.ilp
running ilastik using /home/jmarie/notebooks/pipelines/pixel-class-133.ilp
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to in-memory array.
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 504.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 1.5GiB, split between 2 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (1, 92, 92, 92, 2)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 748.6MiB


WARNING arraytypes.py(1263): FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
WARNING arraytypes.py(1269): FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
ERROR 2021-04-29 11:20:57,791 filterOperators 93 140038725682944    : DifferenceOfGaussiansFF (1, 95, 95, 95) {'sigma0': 0.7, 'sigma1': 0.46199999999999997, 'window_size': 2} False
ERROR 2021-04-29 11:20:57,810 request 93 140038725682944 Traceback (most recent call last):
  File "/srv/conda/envs/notebook/ilastik-meta/ilastik/lazyflow/request/request.py", line 360, in _execute
    self._result = 

ArgumentError: Python argument types in
    vigra.vigranumpycore.constructArrayFromAxistags(type, tuple, numpy.dtype[float32], AxisTags, bool)
did not match C++ signature:
    constructArrayFromAxistags(boost::python::api::object, vigra::ArrayVector<long, std::allocator<long> >, NPY_TYPES, vigra::AxisTags, bool)

###  Compare raw data with the ilastik result.
View the first channel of the original image (left) and the probability map created by pixel classification module of ilastik (right) side-by-side.

In [8]:
compare(first_channel, data[0, 0, :, :, :], shadow=False, gradient_opacity=0.2, ui_collapsed=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

### Compare the original analysis result with the ilastik result.

On the left, the labels loaded from S3 representing the original analysis by the authors of the paper. On the rigth, the probability maps from the pixel classification module of ilastik.

The first 2 z-planes do not have labels.

In [11]:
compare(labels[0, 0, 2:, :, :], data[0, 0, 2:, :, :], shadow=False, gradient_opacity=0.2, ui_collapsed=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

### Overlay the original analysis result and the ilastik result.

In [13]:
names = [(0, 'labels'), (1, 'Pixels Classification')]
viewer = view(labels[0, 0, 2:, :, :],
              label_image=data[0, 0, 2:, :, :],
              label_image_names=names,
              label_image_blend=0.8,
              gradient_opacity=0.5,
              slicing_planes=False)
viewer

Viewer(geometries=[], gradient_opacity=0.5, interpolation=False, label_image_blend=0.8, label_image_names=[(0,…

### License
Copyright (C) 2019-2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.